# Нейронка на gpt-3 для векторизации описания фильмов

# import

In [98]:
from Database.Database import DataFunFilm
import pandas as pd
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
import gensim.downloader as api
import gensim
import re

# Загрузка базы данных

In [2]:
db = DataFunFilm()
db.getTableDF()

,id,kinopoisk_id,imdb_id,name_ru,name_original,poster_url,rating_imdb,rating_imdb_vote_count,rating_film_critics,rating_critics_vote_count,year,film_length,genres,web_url,slogan,description,rating_kinopoisk,short_desription,rating_age_limits
0,1,1043658,tt4202274,Шоу Грязного Фрэнка,The Filthy Frank Show,https://kinopoiskapiunofficial.tech/images/pos...,9.7,32929,NaN,0,2011.0,12.0,фэнтези комедия музыка короткометражка,https://www.kinopoisk.ru/series/1043658/,None,Шоу повествует о героической борьбе сэра Фрэнс...,9.3,None,None
1,2,1309325,None,Космос,,https://kinopoiskapiunofficial.tech/images/pos...,NaN,0,NaN,0,2019.0,NaN,документальный,https://www.kinopoisk.ru/series/1309325/,None,Астрофизик Нил Деграсс Тайсон встает за штурва...,9.3,Нил Деграсс Тайсон отправляется в мысленное пу...,age18
2,3,77044,tt0108778,Друзья,Friends,https://kinopoiskapiunofficial.tech/images/pos...,9.0,937875,NaN,0,1994.0,22.0,мелодрама комедия,https://www.kinopoisk.ru/series/77044/,Everybody needs Friends!,"Шестеро друзей – Рейчел, Моника, Фиби, Джоуи, ...",9.2,"Шестеро друзей, две квартиры и много юмора. Си...",age16
3,4,972889,tt4926348,BadComedian,None,https://kinopoiskapiunofficial.tech/images/pos...,7.7,793,NaN,0,2011.0,NaN,комедия,https://www.kinopoisk.ru/series/972889/,None,"Юмористическое шоу, высмеивающее плохое кино. ...",9.2,None,age18
4,5,674243,None,Счастливые люди,None,https://kinopoiskapiunofficial.tech/images/pos...,8.8,120,NaN,0,2008.0,55.0,документальный,https://www.kinopoisk.ru/series/674243/,None,Документальный фильм о трудах и днях охотников...,9.2,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66752,66753,83307,tt0041640,Мари из порта,La Marie du port,https://kinopoiskapiunofficial.tech/images/pos...,6.7,395,NaN,0,1950.0,88.0,драма мелодрама,https://www.kinopoisk.ru/film/83307/,None,"Анри Шателяру уже за сорок, он владеет рестора...",7.0,None,None
66753,66754,219369,tt0071859,Путешественник,Mossafer,https://kinopoiskapiunofficial.tech/images/pos...,7.5,1695,NaN,0,1974.0,83.0,драма,https://www.kinopoisk.ru/film/219369/,None,Парнишке-школьнику по имени Кассем из маленько...,7.0,None,None
66754,66755,47614,tt0211372,Сын француза,Le fils du Français,https://kinopoiskapiunofficial.tech/images/pos...,5.0,282,NaN,0,1999.0,107.0,приключения комедия семейный,https://www.kinopoisk.ru/film/47614/,None,У Бенжамена 9 лет нет больше мамы. Он хочет на...,7.0,None,None
66755,66756,977755,tt9378362,Звоните ДиКаприо!,None,https://kinopoiskapiunofficial.tech/images/pos...,6.7,496,NaN,0,2018.0,48.0,драма комедия,https://www.kinopoisk.ru/series/977755/,Что останется после тебя?,Егор Румянцев – звезда популярного сериала про...,7.0,None,age18


In [10]:
table = db.getTableDF()
description = table['description'].values
print(description)

['Шоу повествует о героической борьбе сэра Фрэнсиса Грязного и его верных друзей против зловещего тёмного бога в сюрреалистической «омнивселенной».'
 'Астрофизик Нил\xa0Деграсс Тайсон встает за\xa0штурвал своего «Корабля воображения» и\xa0отправляется в\xa0путешествие по\xa0космосу, продолжив то, на\xa0чём остановился Карл Саган несколько десятилетий назад.'
 'Шестеро друзей – Рейчел, Моника, Фиби, Джоуи, Чендлер и Росс – живут по соседству, вместе убивают время, делятся секретами и иногда очень сильно влюбляются.'
 ...
 'У Бенжамена 9 лет нет больше мамы. Он хочет найти своего отца, золотоискателя в Бразилии. Единственные люди, способные его сопровождать в поездке — две бабушки, Анна и Сюзанна, у которых неприязненные отношения. Эпохальная поездка в перспективе, от которой они не готовы отказаться...'
 'Егор Румянцев – звезда популярного сериала про врачей. Он слетает с катушек от успеха: срывает съемки, путается в любовных связях, всем врет. Румянцев чувствует себя неуязвимым и живет

In [115]:
words = []
for i in description:
  sep = re.split(" ", i)
  words.append(sep)
print(words[0])

['Шоу', 'повествует', 'о', 'героической', 'борьбе', 'сэра', 'Фрэнсиса', 'Грязного', 'и', 'его', 'верных', 'друзей', 'против', 'зловещего', 'тёмного', 'бога', 'в', 'сюрреалистической', '«омнивселенной».']


In [116]:
model = Word2Vec(min_count=1)
model.build_vocab(words)  # prepare the model vocabulary
model.train(words, total_examples=model.corpus_count, epochs=model.epochs)
model.save("word2vec.model")

In [123]:
#print(model.wv.key_to_index)
words_trainded = model.wv.most_similar(positive=words[0], topn=3)
for word in words_trainded:
  print(word)
print()

('религиозных', 0.9585441946983337)
('принцессы', 0.9569653868675232)
('Стивена', 0.9534614682197571)



In [71]:
glove_vectors = gensim.downloader.load('word2vec-ruscorpora-300')
glove_vectors.most_similar('помощь_NOUN')

[('помогать_VERB', 0.5959727764129639),
 ('доврачебный_ADJ', 0.5565364360809326),
 ('содействие_NOUN', 0.5554836392402649),
 ('оказание_NOUN', 0.5444939732551575),
 ('посредством_ADP', 0.5133970975875854),
 ('оказывать_VERB', 0.5025925040245056),
 ('поддержка_NOUN', 0.4895719885826111),
 ('кардиограф_NOUN', 0.45548754930496216),
 ('пострадавший_NOUN', 0.44766050577163696),
 ('ohno_NOUN', 0.4456321597099304)]